In [23]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh
import time
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
%matplotlib notebook

%load_ext tensorboard

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

2.3.0
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [57]:
#load mesh of object
surf_fn = "C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/wall_scaled.stl"
M = trimesh.load(surf_fn)
#get ground truth translation
gt = np.loadtxt("C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/gt.txt")
print(gt)
#load point clouds of surface generated in Matlab
pts_fn = "C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/s1.txt"
pts = np.loadtxt(pts_fn)
pts = pts - gt #+ np.array([20,5,1.72])

[-40.   -40.    -1.72]


In [59]:
#plot with Vedo
plt1 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp = []
disp.append(Points(pts, c = 'red', r = 5))
toilet = Mesh(M).c("gray").alpha(1)
disp.append(toilet)
plt1.show(disp, "Fig 2.1")
ViewInteractiveWidget(plt1.window)

[vedo.plotter.py:3290] INFO: Closing window now. Plotter.escaped is set to True.


ViewInteractiveWidget(height=1377, layout=Layout(height='auto', width='100%'), width=2560)